In [15]:
import tensorflow as tf

In [16]:
import pandas as pd
from collections import Counter

In [17]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import os
import random
import numpy as np
import pickle

In [18]:
from ReLERNN import *

In [90]:

def Count_SNP(window,df1,targets):
    lis2=[]
    targ={}

    for i in df1['Position']:
        if i!=None:
            valu=np.floor(int(i)/window)
            if valu in targ.keys():
                targ[valu]+=targets[int(valu)][0]
            else:
                targ[valu]=targets[int(valu)][0]
            lis2.append(valu)
            
 
    frequ = dict(Counter(lis2))
    
    for i in frequ.keys():
        targ[i]=targ[i]/frequ[i]
        
    return frequ,targ


def count_window(r):
    df1= pd.DataFrame(r.T,columns=['Position'])
    
    respectiveNormalizedTargets = [[t] for t in normalizeTargets()]
    targets = np.array(respectiveNormalizedTargets)
    
    window=10
    cal,targ= Count_SNP(1000,df1,targets)
    idx=0
    lis=[]
    lis2=[]
    for i in cal:
        lis.append([idx,idx+window,cal[i]])
        lis2.append(targ[i])
        idx=idx+window
  

    tar_mean = np.mean(lis2)
    tar_sd = np.std(lis2)
    nTargets =lis2- tar_mean
    nTargets = np.divide(nTargets,tar_sd,out=np.zeros_like(nTargets),where=tar_sd!=0)

    return np.array(lis),np.array(nTargets)


def normalizeTargets(norm='zscore'):
    infoFilename= os.path.join('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/',"info.p")
    nTargets_1 = pickle.load(open(infoFilename,"rb"))['rho']
    print(len(nTargets_1))

    if(norm == 'zscore'):
        tar_mean = np.mean(nTargets_1,axis=0)
        tar_sd = np.std(nTargets_1,axis=0)
        nTargets =nTargets_1- tar_mean
        nTargets = np.divide(nTargets,tar_sd,out=np.zeros_like(nTargets),where=tar_sd!=0)

    return nTargets_1

In [91]:


def __data_generation1(batchTreeIndices,val):

        haps = []
        pos = []
        for treeIndex in batchTreeIndices:
            print(treeIndex)
            Hfilepath = os.path.join('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/'+val+'/val/',str(treeIndex) + "_haps.npy")
            Pfilepath = os.path.join('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/'+val+'/val/',str(treeIndex) + "_pos.npy")
            print(Hfilepath)
            H = np.load(Hfilepath)
            P = np.load(Pfilepath)
            haps.append(H)
            pos.append(P)

        respectiveNormalizedTargets = [[t] for t in normalizeTargets()]
        targets = np.array(respectiveNormalizedTargets)
       
        return haps,pos


            
def __getitem__1(idx,val):
    X,x1= __data_generation1([idx],val)
    return X,x1


In [92]:
os.environ['PYTHONHASHSEED']=str(42)
random.seed(42)
np.random.seed(42)
#nProc = mp.cpu_count()
   

projectDir = '/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool'
trainDir = os.path.join(projectDir,"train")
valiDir = os.path.join(projectDir,"vali")
testDir = os.path.join(projectDir,"test")
networkDir = os.path.join(projectDir,"networks")


test_resultFile = os.path.join(networkDir,"testResults.p")
test_resultFig = os.path.join(networkDir,"testResults.pdf")
modelSave = os.path.join(networkDir,"model.json")
weightsSave = os.path.join(networkDir,"weights.h5")


maxSimS = 0
winFILE=os.path.join(networkDir,"windowSizes.txt")
with open(winFILE, "r") as fIN:
    for line in fIN:
        maxSimS=max([maxSimS, int(line.split()[5])])
maxSegSites = 0

for ds in [trainDir,valiDir,testDir]:
    DsInfoDir = pickle.load(open(os.path.join(ds,"info.p"),"rb"))
    segSitesInDs = max(DsInfoDir["segSites"])
    maxSegSites = max(maxSegSites,segSitesInDs)
maxSegSites = max(maxSegSites, maxSimS)




bds_train_params = {
    'treesDirectory':trainDir,
    'targetNormalization':"zscore",
    'batchSize': 64,
    'maxLen': 448,
    'frameWidth': 5,
    'shuffleInds':True,
    'sortInds':False,
    'center':False,
    'ancVal':-1,
    'padVal':0,
    'derVal':1,
    'realLinePos':True,
    'posPadVal':0,
    'seqD':20,
    'maf':0.05,
    'seed':42
            }


x_1,y = SequenceBatchGenerator(**bds_train_params).__getitem__(0)
x,pos= __getitem__1(0,'train')


vale1,targets = count_window(pos[0])

value = tf.convert_to_tensor(vale1,dtype=None)


value.shape
xTrain = np.reshape(value, (value.shape[0], value.shape[1], 1))
sites= xTrain.shape
genotype_inputs = layers.Input(shape=(xTrain.shape[1],xTrain.shape[2]))

model1 = layers.Bidirectional(layers.GRU(84,return_sequences=False,input_shape=(1348,3)))(genotype_inputs)
model1 = layers.Dense(256)(model1)
model1 = layers.Dropout(0.35)(model1)
output = layers.Dense(1)(model1)

model1 = Model(inputs=genotype_inputs, outputs=[output])
model1.compile(optimizer='Adam', loss='mse')
model1.summary()





0
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/0_haps.npy
13000
13000
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 3, 1)]            0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 168)              43848     
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 256)               43264     
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 1)                 257       
                                                         

In [35]:
vale1


array([[   0,   10,   17],
       [  10,   20,   15],
       [  20,   30,   15],
       [  30,   40,   17],
       [  40,   50,   11],
       [  50,   60,    7],
       [  60,   70,    4],
       [  70,   80,   13],
       [  80,   90,   12],
       [  90,  100,   10],
       [ 100,  110,   25],
       [ 110,  120,   11],
       [ 120,  130,   11],
       [ 130,  140,   12],
       [ 140,  150,    4],
       [ 150,  160,    8],
       [ 160,  170,    7],
       [ 170,  180,    4],
       [ 180,  190,   14],
       [ 190,  200,   12],
       [ 200,  210,   15],
       [ 210,  220,   15],
       [ 220,  230,    8],
       [ 230,  240,   14],
       [ 240,  250,   16],
       [ 250,  260,    7],
       [ 260,  270,   16],
       [ 270,  280,   11],
       [ 280,  290,   14],
       [ 290,  300,   18],
       [ 300,  310,   13],
       [ 310,  320,   23],
       [ 320,  330,   18],
       [ 330,  340,   21],
       [ 340,  350,   10],
       [ 350,  360,    9],
       [ 360,  370,   14],
 

In [93]:
vali_x=[]
vali_y=[]
for i in range(1000):
    x,pos= __getitem__1(i,'vali')

    vale1,targets = count_window(pos[0])


    value = tf.convert_to_tensor(vale1,dtype=None)



    value.shape
    xvali = np.reshape(value, (value.shape[0], value.shape[1], 1))
    vali_x.append(xvali)
    vali_y.append(targets)

0
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/0_haps.npy
13000
13000
1
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/1_haps.npy
13000
13000
2
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/2_haps.npy
13000
13000
3
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/3_haps.npy
13000
13000
4
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/4_haps.npy
13000
13000
5
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/5_haps.npy
13000
13000
6
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/6_haps.npy
13000
13000
7
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-mast

13000
13000
63
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/63_haps.npy
13000
13000
64
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/64_haps.npy
13000
13000
65
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/65_haps.npy
13000
13000
66
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/66_haps.npy
13000
13000
67
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/67_haps.npy
13000
13000
68
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/68_haps.npy
13000
13000
69
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/69_haps.npy
13000
13000
70
/N/u/samishr/Carbonate/Desktop/Recombina

13000
13000
124
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/124_haps.npy
13000
13000
125
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/125_haps.npy
13000
13000
126
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/126_haps.npy
13000
13000
127
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/127_haps.npy
13000
13000
128
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/128_haps.npy
13000
13000
129
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/129_haps.npy
13000
13000
130
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/130_haps.npy
13000
13000
131
/N/u/samishr/Carbonate/De

13000
13000
184
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/184_haps.npy
13000
13000
185
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/185_haps.npy
13000
13000
186
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/186_haps.npy
13000
13000
187
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/187_haps.npy
13000
13000
188
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/188_haps.npy
13000
13000
189
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/189_haps.npy
13000
13000
190
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/190_haps.npy
13000
13000
191
/N/u/samishr/Carbonate/De

13000
13000
245
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/245_haps.npy
13000
13000
246
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/246_haps.npy
13000
13000
247
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/247_haps.npy
13000
13000
248
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/248_haps.npy
13000
13000
249
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/249_haps.npy
13000
13000
250
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/250_haps.npy
13000
13000
251
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/251_haps.npy
13000
13000
252
/N/u/samishr/Carbonate/De

...

13000
733
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/733_haps.npy
13000
13000
734
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/734_haps.npy
13000
13000
735
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/735_haps.npy
13000
13000
736
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/736_haps.npy
13000
13000
737
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/737_haps.npy
13000
13000
738
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/738_haps.npy
13000
13000
739
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/739_haps.npy
13000
13000
740
/N/u/samishr/Carbonate/Desktop/

13000
13000
795
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/795_haps.npy
13000
13000
796
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/796_haps.npy
13000
13000
797
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/797_haps.npy
13000
13000
798
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/798_haps.npy
13000
13000
799
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/799_haps.npy
13000
13000
800
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/800_haps.npy
13000
13000
801
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/801_haps.npy
13000
13000
802
/N/u/samishr/Carbonate/De

855
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/855_haps.npy
13000
13000
856
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/856_haps.npy
13000
13000
857
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/857_haps.npy
13000
13000
858
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/858_haps.npy
13000
13000
859
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/859_haps.npy
13000
13000
860
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/860_haps.npy
13000
13000
861
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/861_haps.npy
13000
13000
862
/N/u/samishr/Carbonate/Desktop/Recomb

13000
917
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/917_haps.npy
13000
13000
918
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/918_haps.npy
13000
13000
919
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/919_haps.npy
13000
13000
920
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/920_haps.npy
13000
13000
921
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/921_haps.npy
13000
13000
922
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/922_haps.npy
13000
13000
923
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/923_haps.npy
13000
13000
924
/N/u/samishr/Carbonate/Desktop/

978
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/978_haps.npy
13000
13000
979
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/979_haps.npy
13000
13000
980
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/980_haps.npy
13000
13000
981
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/981_haps.npy
13000
13000
982
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/982_haps.npy
13000
13000
983
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/983_haps.npy
13000
13000
984
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/984_haps.npy
13000
13000
985
/N/u/samishr/Carbonate/Desktop/Recomb

In [94]:
vali_x[0]

array([[[   0],
        [  10],
        [   8]],

       [[  10],
        [  20],
        [   5]],

       [[  20],
        [  30],
        [  10]],

       [[  30],
        [  40],
        [   6]],

       [[  40],
        [  50],
        [   9]],

       [[  50],
        [  60],
        [   8]],

       [[  60],
        [  70],
        [  16]],

       [[  70],
        [  80],
        [  18]],

       [[  80],
        [  90],
        [   7]],

       [[  90],
        [ 100],
        [  13]],

       [[ 100],
        [ 110],
        [  13]],

       [[ 110],
        [ 120],
        [   7]],

       [[ 120],
        [ 130],
        [  17]],

       [[ 130],
        [ 140],
        [  14]],

       [[ 140],
        [ 150],
        [  10]],

       [[ 150],
        [ 160],
        [  17]],

       [[ 160],
        [ 170],
        [   7]],

       [[ 170],
        [ 180],
        [  14]],

       [[ 180],
        [ 190],
        [  14]],

       [[ 190],
        [ 200],
        [  29]],



In [95]:
callbacks_list = [
    EarlyStopping(
    monitor='val_loss',
    verbose=1,
    min_delta=0.01,
    patience=100),
    ModelCheckpoint(
    filepath='/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val',
    monitor='val_loss',
    save_best_only=True),
    TerminateOnNaN()]


In [102]:
for i in range(25):
    x,pos= __getitem__1(i,'train')


    
    vale1,targets = count_window(pos[0])


    value = tf.convert_to_tensor(vale1,dtype=None)
   
    
    xTrain = np.reshape(value, (value.shape[0], value.shape[1], 1))
    history = model1.fit(xTrain,targets,
    steps_per_epoch=10,
    epochs=1,
    callbacks=callbacks_list,
    validation_data=(vali_x[0],vali_y[0]),
    use_multiprocessing=True)
    #model1.fit(xTrain,targets,epochs=5)
    
    
model_json = model1.to_json()
with open('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/model', "w+") as json_file:
    json_file.write(model_json)



x,pos= __getitem__1(0,'test')
vale1,targets = count_window(pos[0])

value1 = tf.convert_to_tensor(vale1,dtype=None)
   

xTest = np.reshape(value1, (value1.shape[0], value1.shape[1], 1))
predictions = model1.predict(xTest)

history.history['loss'] = np.array(history.history['loss'])
#history.history['val_loss'] = np.array(history.history['val_loss'])
history.history['predictions'] = np.array(predictions)
history.history['Y_test'] = np.array(targets)
history.history['name'] = 'ModelName'

pickle.dump(history.history, open( 'resultsFile', "wb" ))

0
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/0_haps.npy
13000
13000
10/10 [==============================] - 0s 13ms/step - loss: 1.3116 - val_loss: 1.0287
1
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/1_haps.npy
13000
13000
10/10 [==============================] - 0s 13ms/step - loss: 1.2929 - val_loss: 0.9802
2
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/2_haps.npy
13000
13000
10/10 [==============================] - 0s 13ms/step - loss: 0.9932 - val_loss: 0.9850
3
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/3_haps.npy
13000
13000
10/10 [==============================] - 0s 13ms/step - loss: 1.0161 - val_loss: 0.9887
4
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/4

INFO:tensorflow:Assets written to: /N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/assets


INFO:tensorflow:Assets written to: /N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/assets


10/10 [==============================] - 11s 1s/step - loss: 1.0993 - val_loss: 0.9699
6
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/6_haps.npy
13000
13000
 1/10 [==>...........................] - ETA: 0s - loss: 1.0464

INFO:tensorflow:Assets written to: /N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/assets


INFO:tensorflow:Assets written to: /N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/assets


10/10 [==============================] - 11s 1s/step - loss: 1.0526 - val_loss: 0.9696
7
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/7_haps.npy
13000
13000
10/10 [==============================] - 0s 12ms/step - loss: 1.0419 - val_loss: 0.9805
8
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/8_haps.npy
13000
13000
10/10 [==============================] - 0s 11ms/step - loss: 1.0459 - val_loss: 1.0609
9
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/9_haps.npy
13000
13000
10/10 [==============================] - 0s 13ms/step - loss: 1.0292 - val_loss: 0.9728
10
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/10_haps.npy
13000
13000
10/10 [==============================] - 0s 11ms/step - loss: 1.0910 - val_loss: 0.9699
11
/N/u/samishr/Carbona

INFO:tensorflow:Assets written to: /N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/assets


INFO:tensorflow:Assets written to: /N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/assets


10/10 [==============================] - 10s 1s/step - loss: 1.0940 - val_loss: 0.9610
23
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/23_haps.npy
13000
13000
10/10 [==============================] - 0s 11ms/step - loss: 1.0831 - val_loss: 1.0609
24
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/24_haps.npy
13000
13000
10/10 [==============================] - 0s 12ms/step - loss: 1.0119 - val_loss: 0.9647
0
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/test/val/0_haps.npy
13000
13000


In [106]:
erro=[]
for i in range(1000):
    erro.append(np.linalg.norm((model1.predict(vali_x[i])-vali_y[i])*div))

In [109]:
de1= pd.DataFrame(erro)

In [97]:
div=  1/(4*ne)

In [66]:
div

array([272.11637365, 531.98701413, 275.87078574, ..., 316.59039428,
       306.42442032, 305.97317744])

In [110]:
de1

,0
0,0.000491
1,0.000484
2,0.000495
3,0.000495
4,0.000495
...,...
995,0.000495
996,0.000495
997,0.000495
998,0.000495


In [111]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.clf()
sns.boxplot(data=de1)

plt.savefig("/N/u/samishr/Carbonate/Desktop/Recombination_landscape/Data_Processing/fig.png")

In [56]:
infoFilename= os.path.join('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/',"info.p")
ne = pickle.load(open(infoFilename,"rb"))['Ne']

In [51]:
nTargets_1

69982.4499438376

In [55]:
mu = pickle.load(open(infoFilename,"rb"))['mu']

In [53]:
nTargets_1

{'N': 20,
 'Ne': 69982.4499438376,
 'priorLowsRho': 0.0,
 'priorHighsRho': 1e-08,
 'priorLowsMu': 6.6e-09,
 'priorHighsMu': 1.3300000000000002e-08,
 'ChromosomeLength': 136200,
 'MspDemographics': None,
 'rho': array([3.74540119e-09, 9.50714306e-09, 7.31993942e-09, ...,
        8.67618866e-09, 3.66456646e-09, 4.32136995e-10]),
 'hotWin': None,
 'mu': array([1.31279282e-08, 6.71505904e-09, 1.29492661e-08, ...,
        1.12837416e-08, 1.16580924e-08, 1.16752855e-08]),
 'segSites': array([1966,  909, 1761, ..., 1459, 1690, 1989]),
 'winMasks': [[0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0513950073421439, [[0, 7000]], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.06607929515418502, [[0, 9000]], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.25697503671071953, [[0, 35000]], 0],
  [0.0, [], 0],
  [0.02936857562408223, [[0